In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Predictive_Maintenance_JetEngine'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    x_train_data_path: Path
    y_train_data_path: Path
    model_name: str
    params: dict

In [3]:
from PredictiveMaintenance.constants import CONFIG_YAML_FILE_PATH, PARAMS_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
from PredictiveMaintenance.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_YAML_FILE_PATH,
        params_filepath = PARAMS_YAML_FILE_PATH,
        schema_filepath = SCHEMA_YAML_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostRegressor

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            x_train_data_path=config.x_train_data_path,
            y_train_data_path=config.y_train_data_path,
            model_name=config.model_name,
            params=params
        )

        return model_trainer_config

In [5]:
import pandas as pd
import joblib
from PredictiveMaintenance import logger
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        X_train = pd.read_csv(self.config.x_train_data_path)
        y_train = pd.read_csv(self.config.y_train_data_path)
        logger.info(X_train.head())
        logger.info(y_train.head())

        catboost_regressor = CatBoostRegressor(**self.config.params)
        catboost_regressor.fit(X_train, y_train)

        joblib.dump(catboost_regressor, os.path.join(self.config.root_dir, self.config.model_name))

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    obj = ModelTrainer(config=model_trainer_config)
    obj.train()
except Exception as e:
    raise e

[2024-03-06 07:30:03,923: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-06 07:30:03,925: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-06 07:30:03,929: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-06 07:30:03,930: INFO: common: created directory at: artifacts]
[2024-03-06 07:30:03,931: INFO: common: created directory at: artifacts/model_trainer]
[2024-03-06 07:30:03,987: INFO: 2469689282:    Total LPC outlet temperature  Total HPC outlet temperature  \
0                     -0.972222                      0.101466   
1                     -0.513889                      0.340474   
2                     -0.236111                     -0.091319   
3                     -0.236111                     -0.677565   
4                     -0.208333                     -0.670800   

   Total LPT outlet temperature  Total bypass-duct pressure  \
0                     -0.365599                         0.0   
1                   